## Segway Learning CBF Script

In [1]:
# Imports

# Data Processing
from numpy import array, concatenate, dot, genfromtxt, savetxt, shape
from numpy.random import permutation

# Learning
#from tensorflow.logging import ERROR, set_verbosity
#set_verbosity(ERROR)
from keras import Model, Sequential
from keras.layers import Add, Dense, Dot, Input, Reshape

# Plotting
from matplotlib.pyplot import cla, clf, figure, grid, legend, plot, savefig, show, subplot, title, xlabel, ylabel


In [2]:
# Load Data

# Data will be read in from a .csv file containing data from all episodes concatenated, and stored as a numpy array
# Each line of data will consist of (x, theta, x_dot, theta_dot, dh_dx, dh_dtheta, dh_dxdot, dh_dthetadot, u, hdot_r)

# Read from .csv file
data = genfromtxt('../learning_data.csv', delimiter=',')

# Split data
model_input_data = data[:, 0:8]
control_data = data[:, 8]
h_r_data = data[:, 9]

In [3]:
# Configure Model

# Model parameters
d_drift_in = 8 # dimension of drift model input
d_act_in = 8 # dimension of actuation model input
d_hidden= 400 # number of nodes in hidden layer
d_out = 1 # dimension of combined model output
m = 1 # dimension of control inputs

# Training Parameters
optimizer = 'adam'
loss = 'mean_absolute_error'
batch_size = 1
num_epochs = 10
validation_split = 0.1

# Drift Model
drift_model = Sequential() # Model structure
drift_model.add(Dense(d_hidden, input_shape=(d_drift_in,), activation='relu')) # Hidden layer
drift_model.add(Dense(d_out)) # Output layer
drift_inputs = Input((d_drift_in,)) # Define input to model
drift_residuals = drift_model(drift_inputs) # Define input to 

# Actuation Model
act_model = Sequential()
act_model.add(Dense(d_hidden, input_shape=(d_act_in,), activation='relu'))
act_model.add(Dense(d_out * m))
act_model.add(Reshape((d_out, m)))
act_inputs = Input((d_act_in,))
act_residuals = act_model(act_inputs)

us = Input((m,))

residuals = Add()([drift_residuals, Dot([2, 1])([act_residuals, us])])
model = Model([drift_inputs, act_inputs, us], residuals)
model.compile(optimizer, loss)


In [4]:
# Shuffle Data

perm = permutation(len(h_r_data)) # permutation order

model_input_data_perm = model_input_data[perm]
control_data_perm = control_data[perm]
h_r_data_perm = h_r_data[perm]

In [5]:
# Train Model

model.fit([model_input_data_perm, model_input_data_perm, control_data_perm], 
          h_r_data_perm, batch_size = batch_size, epochs = num_epochs, validation_split = validation_split)

Epoch 1/10
18671/18671 [==============================] - 19s 993us/step - loss: 0.1758 - val_loss: 0.1179
Epoch 2/10
18671/18671 [==============================] - 18s 972us/step - loss: 0.1229 - val_loss: 0.1123
Epoch 3/10
18671/18671 [==============================] - 19s 1ms/step - loss: 0.1091 - val_loss: 0.0980
Epoch 4/10
18671/18671 [==============================] - 19s 1ms/step - loss: 0.1005 - val_loss: 0.0905
Epoch 5/10
18671/18671 [==============================] - 19s 1ms/step - loss: 0.0954 - val_loss: 0.0910
Epoch 6/10
 9961/18671 [===============>..............] - ETA: 7s - loss: 0.0927

KeyboardInterrupt: 

In [ ]:
# Evaluate Model

drift_predictions = drift_model.predict(model_input_data)
act_predictions = act_model.predict(model_input_data)
h_r_predictions = array([dp + dot(ap, [cd]) for dp, ap, cd, in zip(drift_predictions, act_predictions, control_data)])

figure(figsize=(8, 8))
cla()
plot(h_r_predictions, 'g', linewidth=3, label='Predictions')
plot(h_r_data, 'r', linewidth=3, label='True')
grid()
xlabel('Index (#)', fontsize=16)
ylabel('$h_r$', fontsize=16)
legend(fontsize = 16)
show()

In [ ]:
# Extract Model Parameters & Save

# Layers are stored in *_model.layers, use get_weights() to pull out parameters indexed by layer
# Divide parameter sets by layers and model
drift_l1_params = drift_model.layers[0].get_weights()
drift_l2_params = drift_model.layers[1].get_weights()
act_l1_params = act_model.layers[0].get_weights()
act_l2_params = act_model.layers[1].get_weights()

# Extract drift weights and bias.
drift_w_1 = drift_l1_params[0].T
drift_b_1 = drift_l1_params[1].T
drift_w_2 = drift_l2_params[0].T
drift_b_2 = drift_l2_params[1].T

# Extract actuation weights and bias.
act_w_1 = act_l1_params[0].T
act_b_1 = act_l1_params[1].T
act_w_2 = act_l2_params[0].T
act_b_2 = act_l2_params[1].T

savetxt("../network_parameters/drift_w_1.csv", drift_w_1, delimiter=",")
savetxt("../network_parameters/drift_b_1.csv", drift_b_1, delimiter=",")
savetxt("../network_parameters/drift_w_2.csv", drift_w_2, delimiter=",")
savetxt("../network_parameters/drift_b_2.csv", drift_b_2, delimiter=",")
savetxt("../network_parameters/act_w_1.csv", act_w_1, delimiter=",")
savetxt("../network_parameters/act_b_1.csv", act_b_1, delimiter=",")
savetxt("../network_parameters/act_w_2.csv", act_w_2, delimiter=",")
savetxt("../network_parameters/act_b_2.csv", act_b_2, delimiter=",")
